In [ ]:
import pandas as pd
import codecs
import html2text
from bs4 import BeautifulSoup as BS
import re
import os
from pathlib import Path

## Create a list of files in the target folder to use for extracting information from

In [ ]:
# Creating a list of all the files containing SEC filing forms so we can use it for functions, all we need to do is
# insert the file directory in the function

file_list = []

def file_grabber(directory):
    files = os.listdir(directory)
    for file in files:
        if file.endswith('.htm'):
            file_list.append(file)

file_grabber('data/')

## Extracting the ISIN code from each file and adding to a dataframe

In [ ]:
# Creating a dataframe to add ID information to (filing_ID_list) and creating a function to extract the ISIN ID

# Defining the empty dataframe to be filled
filing_ID_list = []

# Defining the function to pull the ISIN id and file name based on a given list of files (list_of_files)
def ISIN_extractor(list_of_files):
    for file in list_of_files: # Loop over each file in the list
        page = open('data/'+file) # Opening the file from the 'data' folder
        ISIN_code = BS(page.read()).text.lower() # Use BeautifulSoup to pull the html text in and convert to lowercase
    
        ISIN_code = re.findall('(?<=isin).*', ISIN_code, re.DOTALL) # Extract all text found after the 'isin' pattern
    
        ISIN_code = list(set(re.findall('[a-z]{2}\d{5}[a-z\d]{3}\d{2}', str(ISIN_code)))) # Add ISIN code patterns
                                                                                          # found to a list once
    
        filing_ID_list.append((file, ISIN_code)) # Add each file name and ISIN code to a list of lists

In [ ]:
# Run the ISIN code extracting function on the list of files created above
ISIN_extractor(file_list)

In [ ]:
# Convert the list of lists to a dataframe (filing_ID_list)
filing_ID_list = pd.DataFrame(filing_ID_list, columns=['file', 'ISIN_id'])
filing_ID_list = pd.DataFrame(filing_ID_list.ISIN_id.values.tolist(), filing_ID_list.file).add_prefix('ISIN_id_').reset_index()

In [ ]:
# Create a new column to contain a list of additional ISIN id's so we can identify files that may warrant investigation
filing_ID_list['extra_ISIN'] = filing_ID_list[filing_ID_list.columns[2:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

# Removing unnecessary columns and renmaing the ISIN id column
filing_ID_list = filing_ID_list[['file', 'ISIN_id_0', 'extra_ISIN']].rename(columns={'ISIN_id_0':'ISIN_id'})

## Extracting the CUSIP code from each file based on ISIN code and file text

In [ ]:
# Creating a CUSIP id column based on removing additional components (2-digit prefix, 1-digit suffix) from ISIN ID
filing_ID_list['CUSIP_id'] = filing_ID_list['ISIN_id'].str[2:11]

In [ ]:
filing_ID_list

In [ ]:
# Since not every file has an associated ISIN ID, we need to extract missing CUSIP ID's for those files.
# The below function pulls out CUSIP ID values from the list of file names similar to how ISIN ID's were pulled out

CUSIP_list = []

def CUSIP_extractor(list_of_files):
    for file in list_of_files:
        try:
            page = open('data/'+file)
            file_name = BS(page.read()).text.lower()
    
            file_name = re.search('(?<=cusip).*', file_name, re.DOTALL).group()
    
            file_name = re.search('\d{5}[a-z\d]{3}\d', str(file_name)).group() # This pulls out the first pattern match
                                                                               # instead of each pattern match for ISIN
        
        except:
            file_name = 'none'
    
        CUSIP_list.append((file, file_name))

In [ ]:
# Run the function to pull out first CUSIP ID pattern match
CUSIP_extractor(file_list)

In [ ]:
# Convert the CUSIP ID list of lists into a dataframe. Name CUSIP column as 'alternates' to merge with full ID list
CUSIP_list = pd.DataFrame(CUSIP_list, columns=['file', 'CUSIP_id_alternate'])

In [ ]:
# Check to ensure the CUSIP ID list looks good
CUSIP_list

In [ ]:
# Merge the full ID list dataframe with the alternate CUSIP dataframe to fill in missing CUSIP ID values
filing_ID_list = filing_ID_list.merge(CUSIP_list, on='file')

# Fill in the missing CUSIP ID's from the CUSIP alternate column
filing_ID_list.CUSIP_id.fillna(value=filing_ID_list['CUSIP_id_alternate'], inplace=True)

# Drop the alternate CUSIP column since no longer needed
filing_ID_list = filing_ID_list.drop('CUSIP_id_alternate', axis=1)

In [ ]:
# Return ISIN id and CUSIP id to uppercase letters to match the conventional format
filing_ID_list['ISIN_id'] = filing_ID_list.ISIN_id.str.upper()
filing_ID_list['CUSIP_id'] = filing_ID_list.CUSIP_id.str.upper()

In [ ]:
# Check of final Filing ID list dataframe
filing_ID_list

## Determining whether each file is for a fix or float bond and adding to dataframe

In [ ]:
# We want to determine whether the bond note is for a fix or float interest rate. Do do this we create an empty list
# and a function to count the number of times the words 'fix' or 'float' are used in the file text and create a list
# for each file

coupon_count_list = []

def fix_or_float (list_of_files):
    for file in list_of_files:
        page = open('data/'+file)
        file_name = BS(page.read()).text.lower()
        
        fix_count = len(re.findall('fix', file_name))
        
        float_count = len(re.findall('float', file_name))
        
        coupon_count_list.append((file, fix_count, float_count))

In [ ]:
# Apply the word counting function across the list of files
fix_or_float(file_list)

In [ ]:
# Convert the list of fix/float word counts into a dataframe
coupon_type = pd.DataFrame(coupon_count_list, columns = ['file', 'fix_count', 'float_count'])

In [ ]:
# If the count of 'fix' is greater than or equal to 'float' call the bond 'fix', otherwise call it 'float'
coupon_type['coupon_type_code'] = (coupon_type['fix_count'] >= coupon_type['float_count']).astype(str)
coupon_type['coupon_type_code'] = coupon_type['coupon_type_code'].str.replace('True', 'fix')
coupon_type['coupon_type_code'] = coupon_type['coupon_type_code'].str.replace('False', 'float')

In [ ]:
# Select only the two needed columns, file name and whether it is fix or float
coupon_type = coupon_type[['file', 'coupon_type_code']]

In [ ]:
# Quick check to ensure all is well
coupon_type

In [ ]:
# Merge the new coupon_type_code column onto the master dataframe with all the filing information
filing_ID_list = filing_ID_list.merge(coupon_type, on='file')

In [ ]:
# Check of the final filing information dataframe
filing_ID_list

## Exporting the cleaned final file information as .csv

In [ ]:
# Export the final dataframe as a csv file
filing_ID_list.to_csv('data/filing_ID_list.csv')